<a href="https://colab.research.google.com/github/mbogantes/Ulatina/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El primer paso a seguir es iniciar las librerías y colocar los parámetros necesarios para que funcionen bien en la plataforma a elegir. En este caso hemos elegido Google Colab como nuestra plataforma de trabajo debido a que proporciona una opción en la nube con una necesidad mínima de instalación de requisitos.

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
from datetime import datetime

Para esta limpieza de datos vamos a crear un "Parser" de la fecha, antes de iniciar hemos abierto el archivo en Notepad para ojear los formatos y los caracteres que se están utilizando. Dado que la fecha se encuentra en un formato no estándar este "Parser" permitirá generar la fecha correcta a partir de la información

In [4]:
pio.renderers.default = "colab"
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

Ahora vamos a cargar nuestro archivo, especificando cuáles serían las columnas en formato de fecha que queremos. Además vamos a especificar el carácter de separación, ya que no es el que Pandas espera por default.

In [5]:
df = pd.read_csv("/content/data_clean_01.csv", sep= ";",
                 parse_dates=['OffenseDate','CallDateTime'],
                 date_parser=custom_date_parser)

Una vez cargado, vamos a dar un vistazo rápido a las características de las columnas

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CrimeId                10051 non-null  int64         
 1   OriginalCrimeTypeName  10051 non-null  object        
 2   OffenseDate            10051 non-null  datetime64[ns]
 3   CallTime               10051 non-null  object        
 4   CallDateTime           10051 non-null  datetime64[ns]
 5   Disposition            10051 non-null  object        
 6   Address                10051 non-null  object        
 7   City                   9730 non-null   object        
 8   State                  10048 non-null  object        
 9   AgencyId               10051 non-null  object        
 10  Range                  0 non-null      float64       
 11  AddressType            10051 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(8)
memory

Podemos observar de entrada que una de las columnas no contiene ninguna información, por lo que vamos a eliminarla, ya que no nos aporta para poder realizar análisis sobre la misma.

In [8]:
df.drop(columns = ['Range'], inplace = True)

Ahora vamos a revisar que efectivamente la hemos eliminado correctamente. El parámetro "inplace" es peligroso de utilizar ya que sobreescribe la base de datos cargada, lo cual simplifica el trabajo a la hora de realizar exploraciones rápidas como ésta, pero en casos de transformaciones muy largas o bases muy pesadas puede resultar en errores que requerirían iniciar desde el principio.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CrimeId                10051 non-null  int64         
 1   OriginalCrimeTypeName  10051 non-null  object        
 2   OffenseDate            10051 non-null  datetime64[ns]
 3   CallTime               10051 non-null  object        
 4   CallDateTime           10051 non-null  datetime64[ns]
 5   Disposition            10051 non-null  object        
 6   Address                10051 non-null  object        
 7   City                   9730 non-null   object        
 8   State                  10048 non-null  object        
 9   AgencyId               10051 non-null  object        
 10  AddressType            10051 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 863.9+ KB


De la información anterior podemos ver que existen columnas que tienen faltante de datos. Vamos a ejecutar un nuevo comando para asegurarnos de cuáles son.

In [11]:
df.isna().any()

CrimeId                  False
OriginalCrimeTypeName    False
OffenseDate              False
CallTime                 False
CallDateTime             False
Disposition              False
Address                  False
City                      True
State                     True
AgencyId                 False
AddressType              False
dtype: bool

Podemos ver que tanto la columna 'City' como la columna 'State' tienen elementos faltantes. En este caso y considerando que esta información sería importante para el análisis, vamos a eliminar las filas con información faltante. Aproximandamente 300 filas serán eliminadas, lo cual representaría un 3% del total de la información.

In [23]:
for x in df.index:
  if df["City"].isnull()[x]:
    df.drop(x, inplace = True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9730 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CrimeId                9730 non-null   int64         
 1   OriginalCrimeTypeName  9730 non-null   object        
 2   OffenseDate            9730 non-null   datetime64[ns]
 3   CallTime               9730 non-null   object        
 4   CallDateTime           9730 non-null   datetime64[ns]
 5   Disposition            9730 non-null   object        
 6   Address                9730 non-null   object        
 7   City                   9730 non-null   object        
 8   State                  9729 non-null   object        
 9   AgencyId               9730 non-null   object        
 10  AddressType            9730 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 1.1+ MB


Una vez eliminadas estas filas podemos observar que ya no tenemos valores nulos. Ahora vamos a revisar un poco las fechas para ver en qué marco temporal nos estamos ubicando

In [33]:
px.histogram(x=df["OffenseDate"])

Podemos ver que prácticamente todas las fechas están en el año 2016, pero tenemos una línea del 2013. Dado que es altamente irregular que esta fecha esté separada por un lapso de 3 años del resto de la información, vamos a eliminarla.

In [37]:
for x in df.index:
  if df.loc[x, "OffenseDate"].year != 2016:
    df.drop(x, inplace = True)

In [39]:
px.histogram(x=df["OffenseDate"],nbins=12)

Ahora podemos ver que las fechas se encuentran en un marco de tiempo mucho más angosto, de hecho estamos hablando de apenas 7 días entre el 30 de marzo de 2016 y el 5 de Abril de 2016. Si se hubiera incluido el dato de 2013, cualquier análisis de tiempo resultaría con un largo tiempo con valores en 0 que dificultaría la visualización.

Ahora vamos a proceder a revisar los principales valores de algunas de las columnas. Por ejemplo, OriginalCrimeTypeName, la cual podemos ver que en su mayoría corresponde a Paradas de tráfico, las cuales constituyen casi un 10% del total.

In [41]:
df['OriginalCrimeTypeName'].value_counts().head(25)

Traffic Stop          1197
Passing Call           871
Suspicious Person      683
Homeless Complaint     576
22500e                 359
Audible Alarm          265
Muni Inspection        256
Suspicious Vehicle     252
Well Being Check       242
Fight No Weapon        231
Trespasser             193
Noise Nuisance         174
Dw                     165
Auto Boost / Strip     164
Meet W/citizen         160
Poss                   137
Mentally Disturbed     124
Petty Theft            114
Assault / Battery      113
Complaint Unkn          97
22500f                  89
Burglary                80
Rep                     72
Stolen Vehicle          70
H&r Veh Accident        68
Name: OriginalCrimeTypeName, dtype: int64

A la hora de revisar la ciudad nos encontramos que la mayoría de los casos provienen de San Francisco, y existen un par de valores que podrían ser errores de digitación. Dado que son bastante evidentes, vamos a corregirlos por el valor de San Francisco.

In [42]:
df['City'].value_counts().head(25)

San Francisco    9664
Treasure Isla      51
Daly City           5
Yerba Buena         3
Presidio            3
SAN FRANCISCO       1
 S                  1
Brisbane            1
Name: City, dtype: int64

In [47]:
for x in df.index:
  if df.loc[x, "City"] == "SAN FRANCISCO":
    df.loc[x, "City"]  = "San Francisco"

for x in df.index:
  if df.loc[x, "City"] == " S":
    df.loc[x, "City"]  = "San Francisco"


In [48]:
df['City'].value_counts().head(25)

San Francisco    9666
Treasure Isla      51
Daly City           5
Yerba Buena         3
Presidio            3
Brisbane            1
Name: City, dtype: int64

Al revisar nuevamente podemos ver que ya todos los valores tienen datos adecuados

In [49]:
df['State'].value_counts().head(25)

CA    9728
Name: State, dtype: int64

En el caso de Estado, todos corresponden a California

In [50]:
df['Disposition'].value_counts().head(25)

HAN             2589
CIT             1405
GOA             1272
ADV             1129
REP              798
Not recorded     496
ND               423
UTL              383
CAN              353
NOM              324
PAS              170
ABA               97
NCR               82
22                77
ARR               65
ADM               46
INC               17
CRT                2
SFD                1
Name: Disposition, dtype: int64

Para el caso de Disposition podemos ver una serie de códigos que en este momento no tienen mucho sentido. Deberíamos revisar la documentación de la base de datos para poder entender esto un poco más, ya que a simple vista resulta confuso. No hay realmente mucho que podamos realizar en este punto.

In [51]:
df['AgencyId'].value_counts().head(25)

1     9728
CA       1
Name: AgencyId, dtype: int64

Para la Agencia, es el mismo dato que Estado. No nos aporta información adicional.

Para el Address, vamos a revisar los primero 25 para ver la información que nos aporta:

In [68]:
for i in range(25) :
  print(df['Address'][i])

100 Block Of Chilton Av
2300 Block Of Market St
2300 Block Of Market St
500 Block Of 7th St
Beale St/bryant St
16th St/pond St
Berwick Pl/harrison St
Florida St/mariposa St
100 Block Of Genebern Wy
2700 Block Of Folsom St
400 Block Of Mission St
700 Block Of Eddy St
1700 Block Of Harrison St
1800 Block Of Market St
0 Block Of Elgin Pk
Vine Tr/pine St
1600 Block Of 15th St
Jones St/golden Gate Av
100 Block Of South Van Ness Av
300 Block Of Ofarrell St
500 Block Of Valencia St
Larkin St/hayes St
600 Block Of Faxon Av
300 Block Of Potrero Av
500 Block Of Grove St


Esta información podría ser de gran importancia para poder revisar la ubicación de las llamadas, pero es difícil poder acceder a la localización real tal comoo está generada. Afortunadamente existen alternativas, como GoogleMapsAPI que permiten generar ubicación aproximada con información de texto tal como esta. Vamos a utilizar datos de Address, Ciudad y Estado para generar un Query y extraer toda la información posible.

In [63]:

from geopy.geocoders import GoogleV3
AUTH_KEY = ("API de Google") #Eliminada por privacidad
geolocator = GoogleV3(api_key=AUTH_KEY)
AddressLat =[]
AddressLon = []

for x in df.index:
   temp_loc = " ".join([str(df.loc[x, "Address"]), str(df.loc[x, "City"]), str(df.loc[x, "State"]) ])
   loc_data = geolocator.geocode(temp_loc)
   AddressLat.append(loc_data.point.latitude)
   AddressLon.append(loc_data.point.longitude)


In [64]:
df['Latitude']=AddressLat
df['Longitude']=AddressLon

Ahora vamos a revisar cuál es la ubicación de nuestras llamadas

In [85]:
# Create scatter map
fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', color = 'OriginalCrimeTypeName')
fig.update_geos(fitbounds="locations", showsubunits=True)
fig.show()

In [109]:
fig = px.scatter_geo(df[df['OriginalCrimeTypeName']=='Traffic Stop'], lat='Latitude', lon='Longitude', scope='usa',title="Crímenes por paradas de tráfico",template='seaborn')
fig.update_geos(fitbounds="locations", showsubunits=True)
fig.show()

Con esto ya podríamos tener una mejor idea de cómo están distribuidos los crímenes. Posteriores análisis por cada tipo de crimen podrían darnos mayor información para poder realizar análisis más profundos de criminalidad.

In [101]:
px.histogram(x=df["CallDateTime"].dt.hour,nbins=48)

Ahora si regresamos a la hora de las llamadas podemos observar cómo las mismas se encuentran distribuidas, y curiosamente la menor cantidad de llamas se da durante la madrugada. Con esto también podríamos revisar en qué lugares salen llamadas a diferentes horas del día

In [110]:
fig = px.scatter_geo(lat=df['Latitude'], lon=df['Longitude'], scope='usa',title="Crímenes por hora",template='seaborn',color=df["CallDateTime"].dt.hour)
fig.update_geos(fitbounds="locations", showsubunits=True)
fig.show()

Finalmente guardamos la base de datos para poder analizarla más profundo en nuevas sesiones

In [65]:
df.to_csv("Cleanedv1.csv")